In [ ]:
import pandas as pd
import pymupdf4llm
import docling
import json
import time

**Using Marker-PDF   **

In [ ]:
from langchain_docling.loader import DoclingLoader

In [ ]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode


In [ ]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser

In [ ]:
import os

In [ ]:
import psutil
import os

# Get the memory usage of the current process (your kernel)
process = psutil.Process(os.getpid())
memory_gb = process.memory_info().rss / (1024 ** 3)  # Convert bytes to GB

print(f"🔹 Current Kernel Memory: {memory_gb:.2f} GB")
print(f"🔹 Total System RAM Used: {psutil.virtual_memory().percent}%")

In [ ]:
!ollama pull qwen2.5:1.5b

In [ ]:
config = {
    "output_format": "markdown",
    "use_llm" : True,
    "llm_service": "marker.services.ollama.OllamaService",
    "llm_model" : "qwen2.5:1.5b",
    "ollama_base_url": "http://localhost:11434"
    }

In [ ]:
converter = PdfConverter(
    config = config,
    artifact_dict = create_model_dict(),
    processor_list = ConfigParser(config).get_processors(),
    renderer = ConfigParser(config).get_renderer(),
    llm_service = ConfigParser(config).get_llm_service()
)

In [ ]:
from pathlib import Path

In [ ]:
path = "p1.pdf"

rendered = converter(path)

In [ ]:
text_1 = "I want to learn NLP!"

In [ ]:
list(text.split())

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from spacy.lang.en.examples import sentences

In [ ]:
doc = nlp(text_1)

In [ ]:
print(doc.text)

In [ ]:
for token in doc:
    print(token.text,token.pos)

In [ ]:
import nltk 
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt_tab')

In [ ]:
tokens = word_tokenize(text_1.lower())

print(tokens)

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
filtered_tokens = [word for word in tokens if word not in stop_words]

In [ ]:
print(filtered_tokens)

In [1]:
from transformers import pipeline

In [15]:
import pymupdf
from pathlib import Path
def summarizer_fn(pdf: Path):
    document = Path(pdf)
    filename = document.stem
    summarizer = pipeline("summarization", model ="facebook/bart-large-cnn")
    summary = {}
    with pymupdf.open(document) as doc:
        for page in doc:
            text = page.get_text()
            text_list = text.split(".")
            Path(f"{filename}_{page}"+".txt").write_bytes(text.encode())
            for c in range(0,len(text_list),200):
                chunk = text_list[c:c+200]
                
                chunk_text = "".join(chunk)
                if len(chunk_text.strip()) < 50:
                    continue
                result = summarizer(chunk_text, max_length = 120, min_length = 30)

                summary[(page, c)] = result[0]['summary_text']


    
    return summary


In [16]:
summarizer_fn("p1.pdf")

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 95bf2759-a733-4b08-aae2-470b2ba2bf83)')' thrown while requesting HEAD https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json
Retrying in 1s [Retry 1/5].


Device set to use cpu


IndexError: index out of range in self